# Preprocess entire and then use existing event_samples to create Epochs anew

In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils
import src.preprocessing.utils_epo as pre_utils_epo 
import mne

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

event_id = {
    "Hand LS": 5,
    "Hand MS": 4,
    "Hand HS": 3,
    "Back LS": 8,
    "Back MS": 7,
    "Back HS": 6,
    "Stop": 9,
    "Pinprick": 10,    
}

for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        
        subject.preprocess(overwrite=True)
        
        # Load epochs info
        subject.load_epochs_info()
        
        # Create epochs
        subject.epochs = mne.Epochs(
            subject.preprocessed_raw,
            subject.event_samples, 
            tmin=-2., 
            tmax=1., 
            baseline=None, 
            preload=True
        )
        
        # Z-score epochs
        subject.epochs = pre_utils_epo.zscore_epochs(subject.epochs)
        
        # Save
        subject.save(subject.epochs, "epochs")
        subject.save(subject.epochs.data, "epochs", as_mat=True)


imported src module
imported preprocessing module
imported configs module
imported utils module
Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/018.EEGDATA/Acquisition 018.Part2 Data.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2959599  =      0.000 ...  2959.599 secs...
Loaded raw for subject 018
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 58 - 62 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) m

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.0s


In [ ]:
subject.epochs